In [4]:
import pandas as pd
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))  # adds parent directory
from experiments_lib import prompt_ilec_data, set_context_window_size, get_context_window, set_base_instructions, get_base_instructions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Actuarial Intern

The intern's manager has described the following task:

>We'd like to select an appropriate expected basis for each insurance plan type. We expect that the basis will vary by issue year, but are open to other ideas.  Which of the bases would provide the best fit, if we are looking at the actual-to-expected ratio on an amount basis? The following expected bases are available:

* Expected_Death_QX7580E_by_Amount
* Expected_Death_QX2001VBT_by_Amount
* Expected_Death_QX2008VBT_by_Amount
* Expected_Death_QX2008VBTLU_by_Amount
* Expected_Death_QX2015VBT_by_Amount
* Expected_Death_QX7580E_by_Policy
* Expected_Death_QX2001VBT_by_Policy
* Expected_Death_QX2008VBT_by_Policy
* Expected_Death_QX2008VBTLU_by_Policy
* Expected_Death_QX2015VBT_by_Policy
* ExpDeathQx2015VBTwMI_byPol
* ExpDeathQx2015VBTwMI_byAmt

This exercise will assume the intern has very little knowledge of the mechanics of life insurance (mortality improvement, preferred classes, etc.), and the final analysis produced by the intern will reflect that.


## Setup LLM params

These are used by the LLM to manage history, the context window includes the last N questions (from the user) and answers (from the agent / LLM)

In [5]:
set_context_window_size(10)

## Canary

In [6]:
print(prompt_ilec_data("How many differerent insurance plan types are there, list them", model="gpt-5"))

Running request...
There is 1 distinct insurance plan type in the data:
- perm_historical


### Modeling

#### Model #1

In [9]:
modeling_prompt = """The goal is to create a model to predict late-duration whole life (perm_historical) mortality on the ILEC_DATA. Use Number_Of_Deaths as the target (y_var) and Expected_Death_QX2015VBT_by_Policy as the offset (offset_var)"""\
    """The ILEC_DATA can be queried with sql_query() and its schema is available with sql_schema(), perform exploratory data analysis with these methods."""\
    """The ILEC_DATA has data quality problems, so identify and fix when possible. Remove any rows where Expected_Death_QX2015VBT_by_Policy <= 0."""\
    """To build a model, first call cmd_init(), this returns a session_id that should be passed as the first argument to the next cmd_*() call."""\
    """Each call to a cmd_*() method returns a session_id, which is an immutable workspace associated with any changes in state resulting from that method call, e.g. creation of datasets, models, and inference data."""\
    """This allows backtracking to previous states in modeling."""\
    """After calling cmd_init(), create two datasets called "perm_data_train" and "perm_data_test" with calls to cmd_create_dataset(), apply fixes for any data quality problems."""\
    """Next, determine variable importance call(s) to cmd_rpart(), use both actuarial soundness and predictive power as criteria for selecting variables."""\
    """Then, call cmd_glmnet() to build a model, any splines specified in design_matrix_vars must have both `Boundary.knots` and `knots` fully specified, do not use the `df` argument."""\
    """Use the decision tree returned by cmd_glmnet() to refine the design matrix, particularly inner knot locations."""\
    """Then run inference using cmd_run_inference() on both perm_data_train and perm_data_test to create perm_data_*_preds.  Run a decision tree on the results using cmd_rpart() and evaluate the final results."""\
    """Back-track if necessary.  Provide a final summary with a short description of the model rationale, any data quality problems and adjustments, and the session_id associated with the final step in the modeling process."""


print(prompt_ilec_data(modeling_prompt))

Running request...
Plan
- Target: Number_Of_Deaths
- Offset: Expected_Death_QX2015VBT_by_Policy (column expdeathqx2015vbtwmi_bypol)
- Scope: dataset = perm_historical, split by train_test
- Remove rows where offset <= 0
- Fix data quality: use attained_age_m = round(issue_age + duration) to standardize age across ALB/ANB and minor rounding inconsistencies.

Key EDA findings
- Rows: 2,701,318 perm_historical (TRAIN 1,901,686; TEST 799,632)
- Offsets: no rows with expdeath<=0; min 1.69e-7, max 27.57
- Ages/duration:
  - issue_age: [0, 98]; attained_age: [4, 113]; duration: [5, 38]
  - Quantiles: issue_age p25/50/75 = 25/39/54; attained_age p25/50/75 = 46/60/75; duration p25/50/75 = 17/22/27
  - 25,400 rows with attained_age < duration (age basis/rounding artifact). We standardized to attained_age_m = round(issue_age + duration)
- Categories: gender {MALE,FEMALE}; smoker_status {NONSMOKER,SMOKER}; age_basis {ANB,ALB}; face_amount_band_int 1..11; preferred_class mostly “NA”; preferred_indi

In [10]:
! cp -r /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/ /home/mike/workspace/soa-ilec/mcp_agent_work_model_1
! rm -rf /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/session_*
! rm -f /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/*

#### Model #2

In [12]:
set_context_window_size(10)

In [13]:
modeling_prompt = """The goal is to create a model to predict late-duration whole life (perm_historical) mortality on the ILEC_DATA. Use Number_Of_Deaths as the target (y_var) and Expected_Death_QX2015VBT_by_Policy as the offset (offset_var)"""\
    """The ILEC_DATA can be queried with sql_query() and its schema is available with sql_schema(), perform exploratory data analysis with these methods."""\
    """The ILEC_DATA has data quality problems, so identify and fix when possible. Remove any rows where Expected_Death_QX2015VBT_by_Policy <= 0."""\
    """To build a model, first call cmd_init(), this returns a session_id that should be passed as the first argument to the next cmd_*() call."""\
    """Each call to a cmd_*() method returns a session_id, which is an immutable workspace associated with any changes in state resulting from that method call, e.g. creation of datasets, models, and inference data."""\
    """This allows backtracking to previous states in modeling."""\
    """After calling cmd_init(), create two datasets called "perm_data_train" and "perm_data_test" with calls to cmd_create_dataset(), apply fixes for any data quality problems."""\
    """We are using Observation_Year for the test-train split. perm_data_train should be Observation_Years < 2016, and perm_data_test should be Observation_Years >= 2016."""\
    """Next, determine variable importance call(s) to cmd_rpart(), use both actuarial soundness and predictive power as criteria for selecting variables."""\
    """Then, call cmd_glmnet() to build a model, any splines specified in design_matrix_vars must have both `Boundary.knots` and `knots` fully specified, do not use the `df` argument."""\
    """Use the decision tree returned by cmd_glmnet() to refine the design matrix, particularly inner knot locations."""\
    """Then run inference using cmd_run_inference() on both perm_data_train and perm_data_test to create perm_data_*_preds.  Run a decision tree on the results using cmd_rpart() and evaluate the final results."""\
    """Back-track if necessary.  Provide a final summary with a short description of the model rationale, any data quality problems and adjustments, and the session_id associated with the final step in the modeling process."""


print(prompt_ilec_data(modeling_prompt))

Running request...
Plan and actions taken
- Explored ILEC_DATA, confirmed target and offset fields, checked data quality, and subset to late-duration whole life: dataset = 'perm_historical'.
- Built train/test datasets by Observation_Year, removed any rows with non-positive offsets (none existed in this dataset), standardized categorical levels.
- Assessed variable importance with a Poisson decision tree (offset exposure), selected variables balancing actuarial sense and predictive power.
- Fit a Poisson GLM via glmnet using natural splines with explicit boundary.knots and knots, then refined spline knot locations based on the model’s post-fit tree diagnostics (back-tracked to re-fit).
- Scored both train and test sets and evaluated with decision trees on the prediction datasets.

Key EDA highlights
- Schema mapping
  - y_var: number_of_deaths
  - offset_var: expdeathqx2015vbtwmi_bypol (Expected_Death_QX2015VBT_by_Policy)
  - Split variable: observation_year
  - Other fields used: gend

In [14]:
! cp -r /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/ /home/mike/workspace/soa-ilec/mcp_agent_work_model_2
! rm -rf /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/session_*
! rm -f /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/*

#### Model #3

In [15]:
set_context_window_size(10)

In [16]:
modeling_prompt = """The goal is to create a model to predict late-duration whole life (perm_historical) mortality on the ILEC_DATA. Use Number_Of_Deaths as the target (y_var) and Expected_Death_QX2015VBT_by_Policy as the offset (offset_var)"""\
    """The ILEC_DATA can be queried with sql_query() and its schema is available with sql_schema(), perform exploratory data analysis with these methods."""\
    """The ILEC_DATA has data quality problems, so identify and fix when possible. Remove any rows where Expected_Death_QX2015VBT_by_Policy <= 0."""\
    """To build a model, first call cmd_init(), this returns a session_id that should be passed as the first argument to the next cmd_*() call."""\
    """Each call to a cmd_*() method returns a session_id, which is an immutable workspace associated with any changes in state resulting from that method call, e.g. creation of datasets, models, and inference data."""\
    """This allows backtracking to previous states in modeling."""\
    """After calling cmd_init(), create two datasets called "perm_data_train" and "perm_data_test" with calls to cmd_create_dataset(), apply fixes for any data quality problems."""\
    """We are using Observation_Year for the test-train split. perm_data_train should be Observation_Years < 2016, and perm_data_test should be Observation_Years >= 2016."""\
    """Next, determine variable importance call(s) to cmd_rpart(), use both actuarial soundness and predictive power as criteria for selecting variables."""\
    """Then, call cmd_glmnet() to build a model, any splines specified in design_matrix_vars must have both `Boundary.knots` and `knots` fully specified, do not use the `df` argument."""\
    """Use the decision tree returned by cmd_glmnet() to refine the design matrix, particularly inner knot locations."""\
    """Then run inference using cmd_run_inference() on both perm_data_train and perm_data_test to create perm_data_*_preds.  Run a decision tree on the results using cmd_rpart() and evaluate the final results."""\
    """Back-track if necessary using information about the errors learned from running inference on the testing set."""\
    """Provide a final summary with a short description of the model rationale, any data quality problems and adjustments, and the session_id associated with the final step in the modeling process."""


print(prompt_ilec_data(modeling_prompt))

Running request...
Plan
- Explore ILEC_DATA structure and quality for perm_historical.
- Create training and test datasets with quality filters and late-duration restriction.
- Screen variable importance with poisson decision trees.
- Fit a Poisson GLM via glmnet with offset = Expected_Death_QX2015VBT_by_Policy.
- Use natural spline terms with explicit inner and boundary knots.
- Run inference on train and test, then tree-based calibration checks (A/E by segments).
- Back-track and refine if errors arise.

Key EDA findings
- Table: perm_historical only, 2,701,318 rows, observation_year 2009–2018.
- No rows with expdeathqx2015vbtwmi_bypol <= 0 (n_bad_expdeath=0).
- Duration: min 5, max 38, avg ~21.7; for obs_year<2016 and duration>=10: min 10, max 35, p10=13, p50=21, p90=29.
- Attained age (train, obs_year<2016, dur>=10): min 9, max 113, p10=32, p50=60, p90=85.
- No nulls in key fields used (number_of_deaths, policies_exposed, amount_exposed). qx has nulls (not used).
- Reasonableness: 

In [17]:
! cp -r /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/ /home/mike/workspace/soa-ilec/mcp_agent_work_model_3
! rm -rf /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/session_*
! rm -f /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/*

## Model #4

In [19]:
set_context_window_size(10)

In [20]:
modeling_prompt = """The goal is to create a model to predict late-duration whole life (perm_historical) mortality on the ILEC_DATA. Use Number_Of_Deaths as the target (y_var) and Expected_Death_QX2015VBT_by_Policy as the offset (offset_var)"""\
    """The ILEC_DATA can be queried with sql_query() and its schema is available with sql_schema(), perform exploratory data analysis with these methods."""\
    """The ILEC_DATA has data quality problems, so identify and fix when possible. Remove any rows where Expected_Death_QX2015VBT_by_Policy <= 0."""\
    """To build a model, first call cmd_init(), this returns a session_id that should be passed as the first argument to the next cmd_*() call."""\
    """Each call to a cmd_*() method returns a session_id, which is an immutable workspace associated with any changes in state resulting from that method call, e.g. creation of datasets, models, and inference data."""\
    """This allows backtracking to previous states in modeling."""\
    """After calling cmd_init(), create two datasets called "perm_data_train" and "perm_data_test" with calls to cmd_create_dataset(), apply fixes for any data quality problems."""\
    """We are using Observation_Year for the test-train split. perm_data_train should be Observation_Years < 2016, and perm_data_test should be Observation_Years >= 2016."""\
    """Next, determine variable importance call(s) to cmd_rpart(), use both actuarial soundness and predictive power as criteria for selecting variables."""\
    """Then, call cmd_glmnet() to build a model, any splines specified in design_matrix_vars must have both `Boundary.knots` and `knots` fully specified, do not use the `df` argument."""\
    """Valid choices of lambda_strat are: 1se, AIC, BIC. Do not include numeric variables in factor_var_levels."""\
    """If you wish to use a numeric variable in factor_var_levels, cast it as a VARCHAR in the dataset creation step cmd_create_dataset()."""\
    """Use the decision tree returned by cmd_glmnet() to refine the design matrix, particularly inner knot locations."""\
    """Then run inference using cmd_run_inference() on both perm_data_train and perm_data_test to create perm_data_*_preds.  Run a decision tree on the results using cmd_rpart() and evaluate the final results."""\
    """Back-track if necessary using information about the errors learned from running inference on the testing set."""\
    """Provide a final summary with a short description of the model rationale, any data quality problems and adjustments, and the session_id associated with the final step in the modeling process."""

print(prompt_ilec_data(modeling_prompt))

Running request...
Plan and progress
- Objective: Predict late-duration whole life (perm_historical) mortality with a Poisson GLMNET using Number_Of_Deaths as y and Expected_Death_QX2015VBT_by_Policy as exposure/offset.
- Data: ILEC_DATA filtered to dataset='perm_historical', split by Observation_Year (<2016 train, >=2016 test), and remove any rows with Expected_Death_QX2015VBT_by_Policy <= 0.

Key exploratory checks
- Rows by dataset: perm_historical only; n=2,701,318; Observation_Year in [2009, 2018], Duration in [5, 38].
- Offset quality: expdeathqx2015vbtwmi_bypol has no zeros or nulls.
- Other quality:
  - qx has ~121,506 nulls; not used as a feature.
  - Numeric exposures are positive: policies_exposed min > 0, amount_exposed min > 0.
  - Categorical levels within limits: gender(2), smoker_status(2), age_basis(2), face_amount_band(11), preferred_class(4), number_of_preferred_classes(3). Some “NA” present as a string level.
- Ranges: issue_age [0,98], attained_age [4,113], duratio

In [21]:
! cp -r /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/ /home/mike/workspace/soa-ilec/mcp_agent_work_model_4
! rm -rf /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/session_*
! rm -f /home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work/*